# A RETRO tutorial by Carson Lam 

this is my notebook to learn and teach the implementation of <a href="https://arxiv.org/abs/2112.04426">RETRO</a>, Deepmind's Retrieval based Attention net, in Pytorch, on a small but meaningful task. 

1. I am using python3.8, if you dont have python 3.8 there are many ways to install it, using pyenv or homebrew, I used homebrew  and followed the instructions at the end of the download. You have to restart your terminal for the changes to take effect.

2. create a virtual environment for this project and entered that environment

```
python3.8 -m venv env
source env/bin/activate
```

3. install this project's dependencies from requirements.txt

```
pip install --upgrade pip
pip install -r requirements.txt
```

4. save any additional dependencies you have pip installed inside your environment along with the specific version back into requirements.txt for later use

```
pip freeze > requirements.txt
```

5. open up jupyter and open this notebook

```
jupyter notebook
```

In [2]:
import torch
from retro_pytorch import RETRO

/media/carson/New Volume/memory/RETRO-pytorch/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
